In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import json
from datetime import datetime
import random

# Base de dados

In [2]:
df = pd.read_excel('../Base120.xlsx')

In [3]:
def filtra_por_solucao(df, vet_solucao):
    '''Recebe um vetor solução e retorna o df filtrado para essa solução'''
    stand_prescription_map = {i + 1: vet_solucao[i] for i in range(len(vet_solucao))}
    filtered_df = df[df.apply(lambda row: row['prescrição'] == stand_prescription_map.get(row['talhao']), axis=1)]
    return filtered_df

def calcula_volume(result):
	"""
	Calculate the volume by summing specific columns of a DataFrame.
	This function takes a DataFrame `result` and returns the sum of the values
	in columns 3 to 18 (inclusive).
	Parameters:
	result (pandas.DataFrame): The input DataFrame containing the data.
	Returns:
	pandas.Series: A Series containing the sum of the specified columns.
	"""
	
    
	return result.iloc[:, 3:19].sum()

def calcula_penalidade(volume_anual):
    '''retorna penalidade por volume excedente/faltante (R$500)'''
    upper = []
    lower = []
    for v_ano in volume_anual:
        if v_ano < 140000:
            lower.append(np.abs(v_ano - 140000))
        elif v_ano > 160000:
            upper.append(np.abs(v_ano - 160000))
    return np.sum(upper + lower)*500

def calcula_vpl_aux(result, volume_ano):
    '''Calcula o VPL penalizado (função auxiliar)'''
    return np.sum(result['VPL']) - calcula_penalidade(volume_ano)

def plota_volume(df, vet_sol):
    '''Plota o gráfico do volume nos anos'''
    volume_ano = filtra_por_solucao(df, vet_sol).iloc[:, 3:19].sum()
    (volume_ano/1000).plot(kind='bar')
    plt.hlines(140, xmin=-1, xmax=16, color='red', alpha=.6)
    plt.hlines(160, xmin=-1, xmax=16, color='red', alpha=.6)
    plt.title("Volume de madeira por ano")
    plt.xlabel("Ano de Produção")
    plt.ylabel("Volume (1000 m$^3$)")

def calcula_volume_aux(df, vet_sol):
	"""
	Calculate the auxiliary volume based on the given solution vector.
	This function filters the DataFrame `df` using the solution vector `vet_sol`
	and then sums the values in columns 3 to 18 (inclusive) of the filtered DataFrame.
	Parameters:
	df (pandas.DataFrame): The input DataFrame containing the data.
	vet_sol (list or array-like): The solution vector used to filter the DataFrame.
	Returns:
	pandas.Series: A Series containing the sum of the values in columns 3 to 18 of the filtered DataFrame.
	"""
    
	return filtra_por_solucao(df, vet_sol).iloc[:, 3:19].sum()

def calcula_vpl_total(df, vet_solucao):
    '''Calcula o VPL total penalizado'''
    result = filtra_por_solucao(df, vet_solucao)
    volume_ano = calcula_volume(result)
    return calcula_vpl_aux(result, volume_ano)

def encontra_pior_ano(df, sol):
    volume_ano = calcula_volume_aux(df, sol)
    return np.argmax(np.abs(volume_ano-160000)) + 1

def numpy_to_python(obj):
    '''Converte np.int e np.float para int e float dos dados do dicionário'''
    if isinstance(obj, (np.integer, np.floating)):
        return obj.item()
    
def python_to_numpy(dicionario):
    '''Converte int e float para np.int e np.float dos dados do dicionário'''
    for chave, subdict in dicionario.items():
        for subchave, valor in subdict.items():
            if isinstance(valor, int):
                subdict[subchave] = np.int64(valor)
            elif isinstance(valor, float):
                subdict[subchave] = np.float64(valor)
                
    dicionario = {np.int64(key): valor for key, valor in dicionario.items()}
    return dicionario

---

## Alteração na base de dados

In [4]:
df_2 = df.drop_duplicates(subset=['talhao'] + df.columns[3:].tolist(), keep='first')
df_2.to_csv('../base_reduzida.csv', index=False)

---

# Algoritmo Genético

## Soluções importadas


In [5]:
with open("../solucoes_alpha.json", "r") as arquivo: # soluções geradas por heurística construtiva
    solucoes_alpha = json.load(arquivo)
solucoes_alpha = [np.array(sol) for sol in solucoes_alpha]

In [6]:
sol = solucoes_alpha[2]
print(calcula_vpl_total(df_2, sol))
random.shuffle(sol)
print(calcula_vpl_total(df_2, sol))

-54932408
-83757683


In [114]:
a={'d':5, 'b':4, 'c':3}

random.sorted(a, 2)

['b', 'c', 'd']

In [285]:
def gera_populacao_inicial(n_pop, n_talhoes=120, perc_aleatoria=0.8):
    populacao = np.random.randint(1, 121, size=(round(n_pop*perc_aleatoria), n_talhoes))  # pop aleatória
    melhor_sol = solucoes_alpha[2]
    
    for _ in range(n_pop-round(n_pop*perc_aleatoria)): # quantidade soluções semi boas
        sol = np.copy(melhor_sol)
        random.shuffle(sol)
        populacao = np.vstack((populacao, sol))
    return populacao

def selecao(populacao, metodo='torneio'):
    pares = np.array([[],[]])
    
    for _ in range(round(populacao.shape[0]/2)):
        pop_torneio = random.sample(list(populacao), round(populacao.shape[0]*0.3))
        p1 = [pop_torneio[np.argmax([calcula_vpl_total(df_2, aux) for aux in pop_torneio])]]
        pop_torneio = random.sample(list(populacao), round(populacao.shape[0]*0.3))
        p2 = [pop_torneio[np.argmax([calcula_vpl_total(df_2, aux) for aux in pop_torneio])]]
        pares = np.append(pares, [[p1], [p2]])  # p1 == p2 !!!!
        
        print(all(p1[0]) == all(p2[0]))
    return pares.reshape(-1, 2, 120)

def recombinacao(pares, prop_corte=0.5):
    filhos = []
    tamanho_sol = round(pares[0][0].shape[0] * prop_corte)
    
    for pai1, pai2 in pares:
        # print(pai1)
        # print(pai2)
        filho1 = np.concatenate((pai1[:tamanho_sol], pai2[tamanho_sol:]))
        filho2 = np.concatenate((pai2[:tamanho_sol], pai1[tamanho_sol:]))
        filhos.append(filho1)
        filhos.append(filho2)
        
    filhos = np.array(filhos)
    pares = pares.reshape(-1, 120)  # Reshape pares to 2 dimensions
    return np.vstack((pares, filhos)).reshape(-1, 120)
        
p = gera_populacao_inicial(10)
pares = selecao(p)
pop_recombinada = recombinacao(pares)


True
True
True
True
True


In [273]:
for a, b in pares:
    print(all(a) == all(b))

True
True
True
True
True
True
True
True
True
True
